In [ ]:
#Class Project (Adam)
#*********************Predicting the helpfulness of Amazon movie product reviews
#*********************My developed classifier assesses a dataset of reviews to predict the helpfulness of reviews
#**************************************************************************************************************
#*********************I will outline my approach to predicting the helpfulness of a review
#*********************the developed classifier will assess reviews randomly from the dataset 
#*********************and predict whether a given product review is either ”helpful” or ”unhelpful.” 
#*********************Will initially use reviewer's star rankings of reviews relative to other reviewers' rankings
#*********************and compare it to helpfulness % /based on responders metrics   
#*********************I was able to achieve up to ___ accuracy on movie reviews.
#   Data Source: Amazon movie reviews past 12 years

In [54]:
#preparing the data and creating a pandas data frame
import pandas as pd
with open(r'/Users/Adam/Documents/DataClass/project/movies_small.txt') as f:
    dict_list = []
    record_dict = {}
    for line in f:
#       print line
        if line.strip() is '':
            dict_list.append(record_dict)
            record_dict = {} # reset record_dict
        else:   
            split_values = line.split(': ',1)
            key = split_values[0]
            try:
                value = split_values[1].rstrip()
            except IndexError:
                value = ''
            record_dict[key] = value

df = pd.DataFrame(dict_list) # store in dataframe
df.head()

,product/productId,review/helpfulness,review/profileName,review/score,review/summary,review/text,review/time,review/userId
0,B003AI2VGA,7/7,"Brian E. Erland ""Rainbow Sphinx""",3.0,"""There Is So Much Darkness Now ~ Come For The ...","Synopsis: On the daily trek from Juarez, Mexic...",1182729600,A141HP4LYPWMSR
1,B003AI2VGA,4/4,Grady Harp,3.0,Worthwhile and Important Story Hampered by Poo...,THE VIRGIN OF JUAREZ is based on true events s...,1181952000,A328S9RN3U5M68
2,B003AI2VGA,8/10,"Chrissy K. McVay ""Writer""",5.0,This movie needed to be made.,The scenes in this film can be very disquietin...,1164844800,A1I7QGUDP043DG
3,B003AI2VGA,1/1,golgotha.gov,3.0,distantly based on a real tragedy,THE VIRGIN OF JUAREZ (2006)<br />directed by K...,1197158400,A1M5405JH9THP9
4,B003AI2VGA,1/1,"KerrLines ""&#34;Movies,Music,Theatre&#34;""",3.0,"""What's going on down in Juarez and shining a ...","Informationally, this SHOWTIME original is ess...",1188345600,ATXL536YX71TR


In [55]:
#split the "review/helpfulness" column into two separate columns of number of reviews and number of helpful reviews
df = df.join(pd.DataFrame(df['review/helpfulness'].str.split('/').tolist(), columns=['num_helpful','num_total']).astype(int))
df.drop('review/helpfulness', inplace=True, axis=1)

In [65]:
# calculate the helpfulness ratio for each review
# convert the "review/score" values from object to float
df['helpful_ratio'] = df['num_helpful']/df['num_total']
df['review/score'] = df['review/score'].astype(float)
df1 = df[['review/score','num_helpful','num_total','helpful_ratio']]
df1 = df1.dropna()
df1.head()

,review/score,num_helpful,num_total,helpful_ratio
0,3,7,7,1.0
1,3,4,4,1.0
2,5,8,10,0.8
3,3,1,1,1.0
4,3,1,1,1.0


In [66]:
import numpy as np
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV
%matplotlib inline

In [67]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31300 entries, 0 to 43389
Data columns (total 4 columns):
review/score     31300 non-null float64
num_helpful      31300 non-null int64
num_total        31300 non-null int64
helpful_ratio    31300 non-null float64
dtypes: float64(2), int64(2)
memory usage: 1.2 MB


In [60]:
# How many rows and columns does the dataset have?
df1.shape

(31300, 4)

In [62]:
# View the correlations
df1.corr()

,review/score,num_helpful,num_total,helpful_ratio
review/score,1.000000,-0.008733,-0.116840,0.250792
num_helpful,-0.008733,1.000000,0.953892,0.137273
num_total,-0.116840,0.953892,1.000000,0.027192
helpful_ratio,0.250792,0.137273,0.027192,1.000000


In [63]:
# Create an empty model
lr = LinearRegression()
# Choose the predictor variables, here all but the first which is the response variable
X = df1['helpful_ratio']
# Choose the response variable(s)
y = df1['review/score']
# Fit the model to the full dataset
lr.fit(X, y)
# Print out the R^2 for the model against the full dataset
lr.score(X,y)

ValueError: Found arrays with inconsistent numbers of samples: [    1 31300]

In [64]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# Fit the model against the training data
lr.fit(X_train, y_train)
# Evaluate the model against the testing data
lr.score(X_test, y_test)

ValueError: Found arrays with inconsistent numbers of samples: [    1 21910]